## Preprocessing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
recipes = pd.read_csv("recipes.csv")
portion_uom_types = pd.read_csv("portion_uom_types.csv")
pos = pd.read_csv("pos_ordersale.csv")
menuitem = pd.read_csv("menuitem.csv")
menu_items = pd.read_csv("menu_items.csv")
ingredients = pd.read_csv("ingredients.csv")
recipe_ingredient_assign = pd.read_csv("recipe_ingredient_assignments.csv")
recipe_sub_recipe = pd.read_csv("recipe_sub_recipe_assignments.csv")
sub_recipe = pd.read_csv("sub_recipes.csv")
sub_recipe_ing = pd.read_csv("sub_recipe_ingr_assignments.csv")

In [3]:
pos_1 = pos [["MD5KEY_ORDERSALE","StoreNumber","date"]]
menuitem_1 = menuitem[["MD5KEY_ORDERSALE", "MD5KEY_MENUITEM","Quantity","Id"]]

In [4]:
merge1 = pd.merge(pos_1, menuitem_1, how = "inner", on = "MD5KEY_ORDERSALE" )

In [5]:
merge1.rename(columns={'Quantity':'order_quantity'}, inplace=True)

In [6]:
menu_items_1 = menu_items[["RecipeId", "MenuItemId"]]

In [7]:
merge2 = pd.merge(merge1,menu_items_1, how = 'left', left_on = 'Id', right_on = 'MenuItemId' )

In [8]:
merge3 = pd.merge(merge2, recipe_ingredient_assign, how = 'left', on = 'RecipeId')

In [9]:
merge4 = pd.merge(merge3, ingredients, how = 'inner', on = 'IngredientId' )

In [10]:
merge5 = pd.merge(merge4, portion_uom_types, how = 'inner', on = 'PortionUOMTypeId')

In [11]:
merge5['Store_name'] = merge5.StoreNumber.map({46673:'California 1 (ID:46673)', 4904:'California 2 (ID:4904)', 12631:'New York 1 (ID:12631)', 20974:'New York 2 (ID:20974)'})

In [12]:
merge2_sub  = pd.merge(merge2, recipe_sub_recipe, how = 'left', on = 'RecipeId')

In [13]:
merge3_sub = pd.merge(merge2_sub, sub_recipe_ing, how = 'left', on = 'SubRecipeId')

In [14]:
merge4_sub = pd.merge(merge3_sub, ingredients, how = 'inner', on = 'IngredientId')

In [15]:
merge5_sub = pd.merge(merge4_sub, portion_uom_types, how = 'inner', on = 'PortionUOMTypeId')

In [16]:
merge5_sub['Store_name'] = merge5_sub.StoreNumber.map({46673:'California 1 (ID:46673)', 4904:'California 2 (ID:4904)', 12631:'New York 1 (ID:12631)', 20974:'New York 2 (ID:20974)'})

In [17]:
Lettuce_main = merge5[merge5.IngredientShortDescription.str.contains('Lettuce')]
Lettuce_sub  = merge5_sub[merge5_sub.IngredientShortDescription.str.contains('Lettuce')]

In [18]:
Lettuce_main['ttl_quantity'] = Lettuce_main['order_quantity'] * Lettuce_main['Quantity']

c:\users\min\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
Lettuce_sub['ttl_quantity'] = Lettuce_sub['order_quantity'] * Lettuce_sub['Quantity'] * Lettuce_sub['Factor']

c:\users\min\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
Lettuce_main_ny1 = pd.DataFrame(Lettuce_main[Lettuce_main.Store_name.apply(lambda x:x == 'New York 1 (ID:12631)')].groupby('date').sum()['ttl_quantity']).reset_index()
Lettuce_main_ny1.rename(columns={'ttl_quantity':'New York 1 (ID:12631)'}, inplace=True)
Lettuce_main_ca1 = pd.DataFrame(Lettuce_main[Lettuce_main.Store_name.apply(lambda x:x == 'California 1 (ID:46673)')].groupby('date').sum()['ttl_quantity']).reset_index()
Lettuce_main_ca1.rename(columns={'ttl_quantity':'California 1 (ID:46673)'}, inplace=True)
Lettuce_main_ny2 = pd.DataFrame(Lettuce_main[Lettuce_main.Store_name.apply(lambda x:x == 'New York 2 (ID:20974)')].groupby('date').sum()['ttl_quantity']).reset_index()
Lettuce_main_ny2.rename(columns={'ttl_quantity':'New York 2 (ID:20974)'}, inplace=True)
Lettuce_main_ca2 = pd.DataFrame(Lettuce_main[Lettuce_main.Store_name.apply(lambda x:x == 'California 2 (ID:4904)')].groupby('date').sum()['ttl_quantity']).reset_index()
Lettuce_main_ca2.rename(columns={'ttl_quantity':'California 2 (ID:4904)'}, inplace=True)

In [21]:
Lettuce_main_df = pd.DataFrame()
Lettuce_main_df['date'] = list(set(Lettuce_main.date))
Lettuce_main_df= Lettuce_main_df.sort_values(by='date')
Lettuce_main_df = Lettuce_main_df.reset_index(drop = True)

In [22]:
Lettuce_main_df = pd.merge(Lettuce_main_df, Lettuce_main_ny1, how = 'left', on = 'date')
Lettuce_main_df = pd.merge(Lettuce_main_df, Lettuce_main_ca1, how = 'left', on = 'date')
Lettuce_main_df = pd.merge(Lettuce_main_df, Lettuce_main_ny2, how = 'left', on = 'date')
Lettuce_main_df = pd.merge(Lettuce_main_df, Lettuce_main_ca2, how = 'left', on = 'date')
Lettuce_main_df.fillna(0, inplace = True)

In [23]:
Lettuce_sub_ny1 = pd.DataFrame(Lettuce_sub[Lettuce_sub.Store_name.apply(lambda x:x == 'New York 1 (ID:12631)')].groupby('date').sum()['ttl_quantity']).reset_index()
Lettuce_sub_ny1.rename(columns={'ttl_quantity':'New York 1 (ID:12631)'}, inplace=True)
Lettuce_sub_ca1 = pd.DataFrame(Lettuce_sub[Lettuce_sub.Store_name.apply(lambda x:x == 'California 1 (ID:46673)')].groupby('date').sum()['ttl_quantity']).reset_index()
Lettuce_sub_ca1.rename(columns={'ttl_quantity':'California 1 (ID:46673)'}, inplace=True)
Lettuce_sub_ny2 = pd.DataFrame(Lettuce_sub[Lettuce_sub.Store_name.apply(lambda x:x == 'New York 2 (ID:20974)')].groupby('date').sum()['ttl_quantity']).reset_index()
Lettuce_sub_ny2.rename(columns={'ttl_quantity':'New York 2 (ID:20974)'}, inplace=True)
Lettuce_sub_ca2 = pd.DataFrame(Lettuce_sub[Lettuce_sub.Store_name.apply(lambda x:x == 'California 2 (ID:4904)')].groupby('date').sum()['ttl_quantity']).reset_index()
Lettuce_sub_ca2.rename(columns={'ttl_quantity':'California 2 (ID:4904)'}, inplace=True)

In [24]:
Lettuce_sub_df = pd.DataFrame()
Lettuce_sub_df['date'] = list(set(Lettuce_sub.date))
Lettuce_sub_df= Lettuce_sub_df.sort_values(by='date')
Lettuce_sub_df = Lettuce_sub_df.reset_index(drop = True)

In [25]:
Lettuce_sub_df = pd.merge(Lettuce_sub_df, Lettuce_sub_ny1, how = 'left', on = 'date')
Lettuce_sub_df = pd.merge(Lettuce_sub_df, Lettuce_sub_ca1, how = 'left', on = 'date')
Lettuce_sub_df = pd.merge(Lettuce_sub_df, Lettuce_sub_ny2, how = 'left', on = 'date')
Lettuce_sub_df = pd.merge(Lettuce_sub_df, Lettuce_sub_ca2, how = 'left', on = 'date')
Lettuce_sub_df.fillna(0, inplace = True)

In [26]:
Lettuce_main_df.head()

,date,New York 1 (ID:12631),California 1 (ID:46673),New York 2 (ID:20974),California 2 (ID:4904)
0,15-03-05,10.0,0.0,0.0,0.0
1,15-03-06,5.0,0.0,0.0,0.0
2,15-03-07,5.0,0.0,0.0,0.0
3,15-03-08,5.0,5.0,0.0,0.0
4,15-03-09,25.0,0.0,0.0,0.0


In [27]:
start_date = '15-03-01'
end_date = '15-06-20'
start = datetime.datetime.strptime(start_date, '%y-%m-%d')
end = datetime.datetime.strptime(end_date, '%y-%m-%d')
step = datetime.timedelta(days=1)

In [28]:
date_list = []
while start <= end:
    date_list.append(start.date())
    start += step

In [29]:
ttl = pd.DataFrame()
ttl['date_new'] = date_list

In [30]:
ttl['date_new'] = pd.DatetimeIndex(ttl.date_new)

In [31]:
for i in range(len(Lettuce_main_df)):
    Lettuce_main_df.loc[i, -1] = datetime.datetime.strptime(Lettuce_main_df['date'][i], "%y-%m-%d")
Lettuce_main_df.rename(columns = {-1:'date_new'}, inplace = True)
Lettuce_main_df = Lettuce_main_df.iloc[:, 1:]

In [32]:
for i in range(len(Lettuce_sub_df)):
    Lettuce_sub_df.loc[i, -1] = datetime.datetime.strptime(Lettuce_sub_df['date'][i], "%y-%m-%d")
Lettuce_sub_df.rename(columns = {-1:'date_new'}, inplace = True)
Lettuce_sub_df = Lettuce_sub_df.iloc[:, 1:]

In [33]:
ttl_data1 = pd.merge(ttl, Lettuce_main_df, how = 'left', left_on = 'date_new', right_on = 'date_new')

In [34]:
ttl_data2 = pd.merge(ttl_data1, Lettuce_sub_df, how = 'left', left_on = 'date_new', right_on = 'date_new')

In [35]:
ttl_data2.dropna(axis = 0, how = 'all', inplace = True)
ttl_data2.fillna(0, inplace=True)

In [36]:
ttl_data2['New York 1 (ID:12631)'] = ttl_data2['New York 1 (ID:12631)_x'] + ttl_data2['New York 1 (ID:12631)_y']
ttl_data2['California 1 (ID:46673)'] = ttl_data2['California 1 (ID:46673)_x'] + ttl_data2['California 1 (ID:46673)_y']
ttl_data2['New York 2 (ID:20974)'] = ttl_data2['New York 2 (ID:20974)_x'] + ttl_data2['New York 2 (ID:20974)_y']
ttl_data2['California 2 (ID:4904)'] = ttl_data2['California 2 (ID:4904)_x'] + ttl_data2['California 2 (ID:4904)_y']

In [37]:
timeserise_data = ttl_data2[['date_new','New York 1 (ID:12631)', 'California 1 (ID:46673)', 'New York 2 (ID:20974)','California 2 (ID:4904)']]

In [38]:
timeserise_data = timeserise_data.iloc[4:107, :].reset_index(drop= True)

In [39]:
timeserise_data
ny1 = timeserise_data[['date_new', 'New York 1 (ID:12631)']]
ca1 = timeserise_data[['date_new', 'California 1 (ID:46673)']]
ny2 = timeserise_data[['date_new', 'New York 2 (ID:20974)']]
ca2 = timeserise_data[['date_new', 'California 2 (ID:4904)']]

In [40]:
len(timeserise_data)

103

In [41]:
timeserise_data

,date_new,New York 1 (ID:12631),California 1 (ID:46673),New York 2 (ID:20974),California 2 (ID:4904)
0,2015-03-05,232.0,152.0,0.0,0.0
1,2015-03-06,227.0,100.0,4.0,0.0
2,2015-03-07,181.0,54.0,0.0,0.0
3,2015-03-08,221.0,199.0,0.0,0.0
4,2015-03-09,229.0,166.0,0.0,0.0
5,2015-03-10,237.0,143.0,0.0,0.0
6,2015-03-11,235.0,162.0,0.0,0.0
7,2015-03-12,244.0,116.0,0.0,0.0
8,2015-03-13,218.0,136.0,29.0,176.0
9,2015-03-14,205.0,68.0,0.0,182.0


In [42]:
timeserise_data.to_csv('datatable.csv')